# Chapter 8: Portfolio Optimization and Advanced Risk Management

## 1. AI-Driven Portfolio Construction: Black-Litterman with ML Enhancements

The Black-Litterman model is a sophisticated method for portfolio optimization that combines market equilibrium with an investor's specific views. AI and machine learning can significantly enhance this model:

- ML for Return Forecasting: Use supervised learning models (e.g., regression, neural networks) to predict asset returns with greater accuracy. These predictions serve as more reliable inputs for the Black-Litterman model.
- NLP for View Generation: Employ Natural Language Processing (NLP) to analyze vast amounts of unstructured data like news articles, social media sentiment, and analyst reports. This allows for the automatic generation of investor "views" and their associated confidence levels, which are crucial components of the Black-Litterman model.
- Reinforcement Learning for Dynamic Adjustments: Train reinforcement learning agents to continuously monitor market conditions and dynamically adjust the portfolio's asset allocation. This creates a self-improving system that adapts to new information and optimizes for long-term rewards.

In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression

# Sample historical returns (features) and future returns (target)
# In a real scenario, you would have a much larger dataset
X_train = np.random.rand(100, 5)  # 100 days, 5 assets
y_train = np.random.rand(100, 5) * 0.01 # Daily returns

# Train a simple linear regression model for each asset
models = [LinearRegression().fit(X_train, y_train[:, i]) for i in range(5)]

# Use the trained models to predict future returns
# These predictions can be used as the 'views' in a Black-Litterman model
predicted_returns = np.array([model.predict(X_train[-1:]) for model in models]).flatten()

print("Predicted Returns for the next period:")
print(predicted_returns)


Predicted Returns for the next period:
[0.00476274 0.00512867 0.0053833  0.00537117 0.0044801 ]


## 2. Dynamic Rebalancing: Trigger-Based and Time-Based Algorithms

Dynamic rebalancing ensures that your portfolio stays aligned with your desired asset allocation. The two primary methods are:

- Time-Based Rebalancing:
How it works: The portfolio is rebalanced at predetermined intervals, such as daily, monthly, or quarterly.
Pros: Simple to implement and maintain.
Cons: Can be inefficient in volatile markets, either by rebalancing too often or not often enough.
- Trigger-Based Rebalancing:
How it works: Rebalancing is triggered only when an asset class deviates from its target allocation by a predefined percentage (e.g., 5%).
Pros: More responsive to market movements and can potentially lead to better performance.
Cons: Can result in higher transaction costs due to more frequent trading in volatile periods.

In [2]:
import pandas as pd

# --- Time-Based Rebalancing ---
def time_based_rebalancing(portfolio_values, target_weights, last_rebalance_date, current_date, frequency='M'):
    """Rebalances the portfolio at a fixed time interval (e.g., monthly)."""
    if pd.to_datetime(current_date).to_period(frequency) > pd.to_datetime(last_rebalance_date).to_period(frequency):
        print(f"Rebalancing portfolio on {current_date}...")
        # In a real implementation, you would execute trades to bring the
        # portfolio back to the target weights.
        return True
    return False

# --- Trigger-Based Rebalancing ---
def trigger_based_rebalancing(current_weights, target_weights, threshold=0.05):
    """Rebalances the portfolio when any asset's weight deviates by more than the threshold."""
    if np.any(np.abs(current_weights - target_weights) > threshold):
        print("Rebalancing portfolio due to weight deviation...")
        # In a real implementation, you would execute trades.
        return True
    return False

# Example Usage
target_weights = np.array([0.4, 0.3, 0.2, 0.1])
current_weights = np.array([0.45, 0.28, 0.17, 0.1])

time_based_rebalancing(None, target_weights, '2023-01-15', '2023-02-01')
trigger_based_rebalancing(current_weights, target_weights)


Rebalancing portfolio on 2023-02-01...


False

## 3. Risk Parity and Alternative Weighting Schemes

Risk parity is a portfolio construction approach that focuses on equalizing the risk contribution of each asset, rather than the capital allocation.

Key Concept: 
Instead of a 60/40 stock/bond portfolio where stocks might contribute 90% of the risk, a risk parity approach would balance the risk contributions of both asset classes.

Benefits:
- Enhanced Diversification: Reduces the concentration of risk in a small number of assets.
- Lower Volatility: A more balanced risk profile can lead to a smoother return stream.
- Improved Risk-Adjusted Returns: By managing risk more effectively, risk parity aims to deliver better returns for the amount of risk taken.


In [3]:
import numpy as np
import pandas as pd

def calculate_risk_parity_weights(returns_df):
    """Calculates risk parity weights based on the inverse of volatility."""
    # Calculate the volatility (standard deviation) of each asset
    volatilities = returns_df.std()

    # The weight of each asset is proportional to the inverse of its volatility
    inverse_volatilities = 1 / volatilities
    risk_parity_weights = inverse_volatilities / np.sum(inverse_volatilities)

    return risk_parity_weights

# Example with dummy returns data
returns_data = {
    'Asset A': np.random.normal(0, 0.01, 100),
    'Asset B': np.random.normal(0, 0.02, 100),
    'Asset C': np.random.normal(0, 0.005, 100),
}
returns_df = pd.DataFrame(returns_data)

weights = calculate_risk_parity_weights(returns_df)
print("Risk Parity Weights:")
print(weights)


Risk Parity Weights:
Asset A    0.304621
Asset B    0.142555
Asset C    0.552824
dtype: float64


## 4. Advanced Risk Controls: Position Sizing, Correlation Monitoring, Tail Risk Management

Effective risk management goes beyond simple diversification. Advanced controls include:

- Position Sizing: Determining the optimal amount of capital to allocate to each position. This can be based on factors like the asset's volatility, the Kelly Criterion, or your own risk tolerance.
- Correlation Monitoring: Actively tracking the correlation between assets in your portfolio. Correlations can change, especially during market stress, and what was once a diversified portfolio can become highly correlated.
- Tail Risk Management: Implementing strategies to protect against rare but extreme market events (i.e., "black swans"). This can involve using options, futures, or other derivatives to hedge against catastrophic losses.

In [4]:
# --- Position Sizing (Volatility-Based) ---
def get_position_size(account_size, risk_per_trade, volatility):
    """Calculates position size based on volatility."""
    return (account_size * risk_per_trade) / volatility

# --- Correlation Monitoring ---
def monitor_correlations(returns_df, threshold=0.8):
    """Monitors the correlation matrix and flags high correlations."""
    corr_matrix = returns_df.corr()
    high_corr = corr_matrix[(corr_matrix > threshold) & (corr_matrix < 1.0)]
    if not high_corr.isnull().all().all():
        print("High Correlation Alert:")
        print(high_corr.dropna(how='all', axis=1).dropna(how='all'))

# --- Tail Risk Management (Simple Stop-Loss) ---
def check_stop_loss(current_price, entry_price, stop_loss_pct=0.1):
    """Checks if a stop-loss has been triggered."""
    if current_price < entry_price * (1 - stop_loss_pct):
        print(f"Stop-loss triggered at {current_price:.2f}!")
        return True
    return False


## 5. Drawdown Management and Recovery Strategies

Drawdown management is about limiting the magnitude of losses during market downturns.

Drawdown Management Techniques:

- Stop-Loss Orders: Automatically sell a security if it falls to a certain price.
- Trailing Stops: A stop-loss order that moves up as the price of the security rises, locking in profits while still protecting against a downturn.

Recovery Strategies:
- Have a Plan: A pre-defined plan for how to act after a significant drawdown.
- Rebalancing: Rebalancing your portfolio can help you buy low and sell high, positioning you for a recovery.
- Scaling into Positions: Gradually adding to positions as the market recovers.

In [5]:
import pandas as pd

def calculate_drawdown(portfolio_returns):
    """Calculates the drawdown of a portfolio."""
    cumulative_returns = (1 + portfolio_returns).cumprod()
    peak = cumulative_returns.expanding(min_periods=1).max()
    drawdown = (cumulative_returns - peak) / peak
    return drawdown

# Example with dummy returns
portfolio_returns = pd.Series(np.random.normal(0.001, 0.02, 252))
drawdowns = calculate_drawdown(portfolio_returns)

print("Max Drawdown:", f"{drawdowns.min()*100:.2f}%")


Max Drawdown: -29.31%


## 6. Multi-Strategy Portfolio Orchestration

This involves combining multiple, distinct trading strategies into a single, unified portfolio.

The Goal: 

To create a portfolio that is more resilient and profitable than any single strategy on its own.

Key Principle: 

- Select strategies with low correlation to each other. For example:
- A trend-following strategy that performs well in trending markets.
- A mean-reversion strategy that excels in range-bound or choppy markets.
- A value-based strategy that focuses on long-term fundamentals.

Benefit: 

The combined portfolio can generate more consistent returns across a wider range of market environments.

In [6]:
import pandas as pd

# Assume you have the daily returns of two different strategies
strategy_A_returns = pd.Series(np.random.normal(0.001, 0.01, 100))
strategy_B_returns = pd.Series(np.random.normal(0.0005, 0.015, 100))

# Combine the strategies into a single portfolio with equal weighting
multi_strategy_portfolio = 0.5 * strategy_A_returns + 0.5 * strategy_B_returns

print("Multi-Strategy Portfolio Performance:")
print(multi_strategy_portfolio.describe())


Multi-Strategy Portfolio Performance:
count    100.000000
mean       0.001414
std        0.009254
min       -0.023763
25%       -0.004260
50%        0.000902
75%        0.006155
max        0.021609
dtype: float64


## 7. Real-Time Risk Monitoring and Automated Circuit Breakers

In today's fast-paced markets, real-time risk monitoring is crucial.

Real-Time Monitoring: 

Use sophisticated analytics to continuously monitor your portfolio's risk exposure using metrics like:
- Value at Risk (VaR): The maximum potential loss over a given time period at a certain confidence level.
- Conditional Value at Risk (CVaR): The expected loss if the VaR threshold is exceeded.

Automated Circuit Breakers: 

These are pre-programmed rules that automatically halt trading or reduce exposure when certain risk thresholds are breached. This can prevent panic selling and protect against catastrophic losses in a market crash.

In [8]:
import numpy as np

def calculate_var(returns, confidence_level=0.95):
    """Calculates the Value at Risk (VaR) of a portfolio."""
    return np.percentile(returns, 100 * (1 - confidence_level))

def check_circuit_breaker(portfolio_returns, var_threshold=-0.05):
    """A simple circuit breaker based on VaR."""
    current_var = calculate_var(portfolio_returns)
    if current_var < var_threshold:
        print(f"CIRCUIT BREAKER TRIPPED! VaR of {current_var:.2%} exceeds threshold.")
        # In a real system, you would halt trading or reduce exposure.
        return True
    return False

# Example with dummy returns
portfolio_returns = np.random.normal(-0.01, 0.03, 1000)
check_circuit_breaker(portfolio_returns)


CIRCUIT BREAKER TRIPPED! VaR of -5.76% exceeds threshold.


True

## 8. Integration with Chapter 4 Quantitative Models and Chapter 6 RL Approaches

This is about creating a holistic and intelligent investment system by combining different AI techniques.

Quantitative Models (from Chapter 4): 

Use these models to generate the fundamental inputs for your portfolio, such as:
- Expected returns for different assets.
- Covariance matrices to model how assets move together.

Reinforcement Learning (from Chapter 6): 

Use RL agents to create a dynamic and adaptive layer on top of your quantitative models. The RL agent can learn from market feedback and:
- Adjust portfolio allocations in real-time.
- Optimize trading execution.
- Develop new, emergent strategies over time.

In [9]:
# --- Inputs ---
# From a quantitative model (e.g., Chapter 4)
quant_expected_returns = np.array([0.02, 0.03, 0.015])

# From a reinforcement learning agent (e.g., Chapter 6)
# The RL agent might suggest adjustments based on market sentiment or momentum
rl_dynamic_weights = np.array([1.1, 0.9, 1.0]) # e.g., overweight asset 1, underweight asset 2

# --- Integration ---
# The final portfolio allocation can be a combination of these inputs
final_expected_returns = quant_expected_returns * rl_dynamic_weights

# These final returns would then be used in a portfolio optimizer
print("Final Expected Returns for Optimization:")
print(final_expected_returns)


Final Expected Returns for Optimization:
[0.022 0.027 0.015]


# Summary

Chapter 8 explores AI-enhanced portfolio optimization, dynamic rebalancing, risk control mechanisms, drawdown management, and multi-strategy orchestration.